# Tableau and player form

Here we create a csv file that is used for making further Tableau visualizations. 

In [1]:
# recent gameweek
gameweek = 24

In [2]:
# import basic libraries
import pandas as pd
import numpy as np

pd.set_option('max_columns',100)

In [3]:
# fetch player stats data
all_df = pd.read_csv('../data/fpl/data_week1.csv', index_col=0)
all_df['gameweek'] = 1
all_df['xPoints'] = all_df['adjusted points']

for week in range(2,gameweek+1):
    filepath = '../data/fpl/data_week' + str(week) + '.csv'
    df = pd.read_csv(filepath, index_col=0)
    df['gameweek'] = week
    column_name = 'xPoints week ' + str(week)
    df['xPoints'] = df[column_name]
    all_df = all_df.append(df,sort='False')

all_df.shape

(14640, 256)

In [4]:
# calculate player form
for i in [5,10]:
    gameweeks = np.arange(gameweek-i+1,gameweek+1)
    column_name = 'form ' + str(i)
    all_df.loc[all_df['gameweek']==gameweek, column_name] = (all_df[all_df['gameweek'].isin(gameweeks)].groupby(['id'])['xPoints'].sum() / (all_df.loc[(all_df['gameweek']==gameweek),'games played'].round() - all_df.loc[(all_df['gameweek']==gameweek-i),'games played'].round()) )

In [5]:
# top form players
all_df[all_df['gameweek']==gameweek][['web_name', 'games played','total_points', 'points_per_game','adjusted points','adjusted points per game',\
                                'form 5', 'form 10']].sort_values(by=['form 5'], ascending=False).head(20)

,web_name,games played,total_points,points_per_game,adjusted points,adjusted points per game,form 5,form 10
id,,,,,,,,
251,Mané,20.980392,107,5.1,118.218179,5.634698,7.881272,5.816691
112,Arrizabalaga,3.947368,15,3.8,17.183247,4.353089,6.986341,6.986341
271,Gündogan,18.088235,123,6.8,105.686570,5.842835,6.974271,6.782841
302,Fernandes,24.054054,178,7.4,160.589790,6.676205,6.915813,6.831543
568,Telles,7.096774,22,3.1,25.030159,3.526977,6.703519,3.851759
466,Neves,22.903226,71,3.1,75.624044,3.301895,6.186610,5.081359
570,Raphinha,18.181818,80,4.4,74.696266,4.108295,6.120215,4.829038
4,Aubameyang,19.787234,93,4.7,94.450781,4.773319,6.116271,6.439594
12,Martínez,22.068966,128,5.8,104.548873,4.737371,6.091122,5.084817


In [6]:
# save data
filepath = '../data/fpl/all_data.csv'
all_df.to_csv(filepath)

filepath = '../data/fpl/data_week' + str(gameweek) + '.csv'
all_df[all_df['gameweek']==gameweek].to_csv(filepath)